# 실력 긍부정

In [12]:
# 실력 긍부정 데이터 불러오기
import pandas as pd

sentence_df = pd.read_csv('./level-lable-2.csv', header=0, sep=',')
sentence_df.head(3)

,sentence,lable
0,"카오스, 아키에이지, 2012년~2014년 리그 오브 레전드, 2016년 오버워치,...",1
1,어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 ...,1
2,게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다.,1


In [13]:
# 전처리 1
import re

sentence_df['sentence'] = sentence_df['sentence'].apply(lambda x : re.sub(pattern="[^ㄱ-ㅎ 가-힣 a-z A-Z]", repl=" ", string=str(x)))

sentence_df.head(10)

,sentence,lable
0,카오스 아키에이지 년 년 리그 오브 레전드 년 오버워치 ...,1
1,어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 ...,1
2,게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다,1
3,이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무...,1
4,당연히 천상계에선 원탑 우디르 유저,1
5,오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 엄...,1
6,롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이 킬 킬...,1
7,플레이 스타일은 피지컬과 반응속도로 찍어누르는 재능충의 표본,1
8,뛰어난 로지컬과 상황 판단력으로 각을 재고 들어가는 타입이 아니라 일단 몸을 던지며...,1
9,의 상황에서도 본인의 피지컬을 신뢰하며 절대 빼지 않고 몸을 던진다,1


In [8]:
# 명사 추출 ( 안하는게 성능 더 좋을수도)

from konlpy.tag import Hannanum

hannanum = Hannanum()
i = 0
for sentence in sentence_df['sentence']:
    original = sentence
    string2noun = hannanum.nouns(original)
    noun2string = " ".join(string2noun)
    sentence_df['sentence'][i] = noun2string
    i += 1

sentence_df.head(10)

,sentence,lable
0,카오스 아키 년 년 리그 오브 레전드 년 오버워치 년 년 배틀그라운드 등 해 인기 ...,1
1,게임 시작 최상위권 달성 실력 재능충 모습 보,1
2,게임 센스 편 화 플레이 편,1
3,이때 잿불거인 전 우디르 고인챔 식 천상계 아무 우디르 우디르 승률 어마어마 수치 ...,1
4,천상계 원탑 우디르 유저,1
5,오버워치 비슷 재능 파카답게 솔 스쿼드,1
6,롤 파카 방송 스쿼드 들 한두 명 킬 멱살잡 우승 버스 기사 역할,1
7,플레이 스타일 피지컬 반응속도 찍어누르 재능충 표본,1
8,로지컬 상황 판단력 각 재고 타입 몸 공격적 포지셔닝,1
9,의 상황 본인 피지컬 신뢰 몸,1


In [20]:
# train-test set 나누기

from sklearn.model_selection import train_test_split

class_df = sentence_df['lable']
feature_df = sentence_df.drop(['lable'],axis=1, inplace=False)



X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.2, random_state=3)

X_train.shape, X_test.shape

((220, 1), (55, 1))

In [21]:
# 로지스틱 회귀 + countVectorize


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['sentence'], y_train)
pred = pipeline.predict(X_test['sentence'])
pred_probs = pipeline.predict_proba(X_test['sentence'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

cnt = 0
for review in X_test['sentence']:
    if pred[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    cnt += 1
    if cnt == 100:
        break

    print('댓글 :{0} 는 {1}로 예상됩니다'.format(review, sentiment))

예측 정확도는 0.7636, ROC_AUC는 0.7619
댓글 :각 챔피언 별 상대 법 갱 호응 및 회피 한 타 운영 법 여타 천상계 유저 에게도 밀리 지 않는다 는 positive로 예상됩니다
댓글 :뉴 슈퍼마리오 브라더스 U 성 클리어 뉴 슈퍼 루이지 U 성 클리어 과제 모드 올 금메달 이라는 엄청난 실력 보여주었다 는 positive로 예상됩니다
댓글 :우수한 실력 탓 여러 커뮤니티 핵 의심 받기도 했으나 당연히 핵 사용 하지 않았고 이를 반박 하기 위해 핵 해명 방송 진행 하여 아예 하나 컨텐츠 만들어 버렸다 는 positive로 예상됩니다
댓글 :점 프맵 실력 매우 좋은 편이 며 컨트롤 상당히 준수 하여 pvp 콘텐츠 등 많은 활약 는 positive로 예상됩니다
댓글 :하지만 멤버 사이 악어 여전히 탑 실력 평가 받고 는 positive로 예상됩니다
댓글 :소환 사 어머니 한편 개리 형 자신 마스터 티어 찍 정도 꽤 실력 자 이지만 프로 게이 머 처럼 피지 컬 좋은 실력 방송 아니라 그냥 재미있게 편안히 볼 있는 방송 추구 는 positive로 예상됩니다
댓글 :시즌 중반 롤 시작 하여 시즌 시즌 까지 모든 시즌 챌 린 저 위 도달 했던 솔 랭 전 사이 자 대리 인성 문제 물의 일으켰고 매우 뛰어난 실력 유명한 아마추어 매 시즌 꾸준히 챌 린 저 위 달성 했던 는 positive로 예상됩니다
댓글 :티어 나 실력 엄연히 최 하위 였던 서새봄 으로서 자신 위치 인정 맞는 역할 충실히 수행 함 으로써 팀 기여 했던 또한 사실 는 positive로 예상됩니다
댓글 :I Wanna Be The Boshy 하드 모드 클리어 는 negative로 예상됩니다
댓글 :특히 운영 형 챔피언 이용 한 플레이 여러 방면 에서의 게임 풀어 나가는 탁월한 능력 거기 암살자 같은 하이퍼 캐리 챔피언 사용 하여 강력한 라인 전 능력 로밍 이용 해 게임 지배 하는 등 여러 부분 다양한 모습 전 천후 번갈아 가면 서 그야말로 만능 가까운 플레이 보여주는 선수 로는 페이커 유일하다

In [22]:
# 로지스틱 회귀 + Tf-idf


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['sentence'], y_train)
pred = pipeline.predict(X_test['sentence'])
pred_probs = pipeline.predict_proba(X_test['sentence'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

cnt = 0
for review in X_test['sentence']:
    if pred[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    cnt += 1
    if cnt == 100:
        break

    print('댓글 :{0} 는 {1}로 예상됩니다'.format(review, sentiment))

예측 정확도는 0.8364, ROC_AUC는 0.7985
댓글 :각 챔피언 별 상대 법 갱 호응 및 회피 한 타 운영 법 여타 천상계 유저 에게도 밀리 지 않는다 는 positive로 예상됩니다
댓글 :뉴 슈퍼마리오 브라더스 U 성 클리어 뉴 슈퍼 루이지 U 성 클리어 과제 모드 올 금메달 이라는 엄청난 실력 보여주었다 는 positive로 예상됩니다
댓글 :우수한 실력 탓 여러 커뮤니티 핵 의심 받기도 했으나 당연히 핵 사용 하지 않았고 이를 반박 하기 위해 핵 해명 방송 진행 하여 아예 하나 컨텐츠 만들어 버렸다 는 positive로 예상됩니다
댓글 :점 프맵 실력 매우 좋은 편이 며 컨트롤 상당히 준수 하여 pvp 콘텐츠 등 많은 활약 는 positive로 예상됩니다
댓글 :하지만 멤버 사이 악어 여전히 탑 실력 평가 받고 는 positive로 예상됩니다
댓글 :소환 사 어머니 한편 개리 형 자신 마스터 티어 찍 정도 꽤 실력 자 이지만 프로 게이 머 처럼 피지 컬 좋은 실력 방송 아니라 그냥 재미있게 편안히 볼 있는 방송 추구 는 positive로 예상됩니다
댓글 :시즌 중반 롤 시작 하여 시즌 시즌 까지 모든 시즌 챌 린 저 위 도달 했던 솔 랭 전 사이 자 대리 인성 문제 물의 일으켰고 매우 뛰어난 실력 유명한 아마추어 매 시즌 꾸준히 챌 린 저 위 달성 했던 는 positive로 예상됩니다
댓글 :티어 나 실력 엄연히 최 하위 였던 서새봄 으로서 자신 위치 인정 맞는 역할 충실히 수행 함 으로써 팀 기여 했던 또한 사실 는 positive로 예상됩니다
댓글 :I Wanna Be The Boshy 하드 모드 클리어 는 positive로 예상됩니다
댓글 :특히 운영 형 챔피언 이용 한 플레이 여러 방면 에서의 게임 풀어 나가는 탁월한 능력 거기 암살자 같은 하이퍼 캐리 챔피언 사용 하여 강력한 라인 전 능력 로밍 이용 해 게임 지배 하는 등 여러 부분 다양한 모습 전 천후 번갈아 가면 서 그야말로 만능 가까운 플레이 보여주는 선수 로는 페이커 유일하다

In [14]:
# 또 다른 시도

import nltk
from konlpy.tag import Okt
t=Okt()

content_text = ""

for sentence in sentence_df['sentence']:
    content_text = content_text + sentence + "\n"

print(content_text)

카오스  아키에이지      년     년 리그 오브 레전드      년 오버워치      년     년 배틀그라운드 등 그 해에 가장 인기를 끌었던 게임마다 파고들면 최상위권을 달성할 정도로 여러 게임을 잘해서 유명해졌다 
어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 보여줬다  
게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다  
이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무도 우디르를 쓰지 않았으나 우디르   판 승률     KDA    이라는 어마어마한 수치로 챌린저에 입성했다 
당연히 천상계에선 원탑 우디르 유저 
오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 엄청난 에임을 보여주고 있다  
롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이  킬   킬을 하거나 빨리 죽어서 없더라도 멱살잡고 우승시켜주는 버스 기사 역할을 하였다 
플레이 스타일은 피지컬과 반응속도로 찍어누르는 재능충의 표본  
뛰어난 로지컬과 상황 판단력으로 각을 재고 들어가는 타입이 아니라 일단 몸을 던지며 최대한 공격적인 포지셔닝을 잡는다 
   의 상황에서도 본인의 피지컬을 신뢰하며 절대 빼지 않고 몸을 던진다 
챔피언 폭도 진 같은 챔피언보다는 베인이나 트위치  코그모 같은 AD하이퍼캐리를 선호하며 그것이 아니라면 케이틀린이나 자야 같은 챔피언을 골라 라인전에서부터 압살을 하려 하는 챔피언을 좋아한다 
딸피로 살아나오는 플레이가 일품 
캐리력이 높은 브루저 챔피언으로 라인전을 터트리고 미드  바텀 순으로 똥을 퍼트리는 전형적인 탑신병자의 캐리 시나리오를 선호하는 모습 
 영웅 모두 화려한 실력으로 노마이크에 솔로큐임에도 불구하고 시즌  경쟁전   승  무   패       로   점을 달성했다 
하지만 예전과 같은 플레이를 보여주지 못해 팟수들의 실망을 자아내었다 
오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를

In [15]:
# 형태소 분석

tokens_ko = t.morphs(content_text)
tokens_ko

['카오스',
 '아키에이지',
 '년',
 '년',
 '리그',
 '오브',
 '레전드',
 '년',
 '오버',
 '워치',
 '년',
 '년',
 '배틀',
 '그',
 '라운드',
 '등',
 '그',
 '해',
 '에',
 '가장',
 '인기',
 '를',
 '끌었던',
 '게임',
 '마다',
 '파고들면',
 '최',
 '상위',
 '권',
 '을',
 '달성',
 '할',
 '정도',
 '로',
 '여러',
 '게임',
 '을',
 '잘',
 '해서',
 '유명해졌다',
 '어떤',
 '게임',
 '이든지',
 '파고들기',
 '시작',
 '하면',
 '최',
 '상위',
 '권',
 '을',
 '달성',
 '하며',
 '엄청난',
 '실력',
 '의',
 '재능',
 '충',
 '모습',
 '을',
 '보여줬다',
 '게임',
 '센스',
 '가',
 '매우',
 '뛰어난',
 '편이',
 '며',
 '화려한',
 '플레이',
 '가',
 '자주',
 '나오는',
 '편이',
 '다',
 '이',
 '때',
 '는',
 '잿불',
 '거인',
 '이',
 '나타나기',
 '전',
 '으로',
 '우디르',
 '가',
 '고인',
 '챔이라는',
 '인식',
 '이',
 '강해',
 '천상계',
 '에서',
 '아무',
 '도',
 '우디르',
 '를',
 '쓰지',
 '않았으나',
 '우디르',
 '판',
 '승률',
 'KDA',
 '이라는',
 '어마어마한',
 '수치',
 '로',
 '챌',
 '린저',
 '에',
 '입성',
 '했다',
 '당연히',
 '천상계',
 '에선',
 '원',
 '탑',
 '우디르',
 '유저',
 '오버',
 '워치',
 '와',
 '비슷한',
 'FPS',
 '게임',
 '에',
 '재능',
 '을',
 '보이는',
 '파카',
 '답',
 '게',
 '솔로',
 '스쿼드',
 '를',
 '가리지',
 '않고',
 '엄청난',
 '에',
 '임',
 '을',
 '보여주고',
 '있다',
 '롤',
 'PD

In [16]:
ko = nltk.Text(tokens_ko)
print(len(ko.tokens))
print(len(set(ko.tokens)))

7817
2397


In [17]:
#빈도수 확인 및 불용어 제거
ko.vocab().most_common(100)

[('을', 293),
 ('이', 186),
 ('를', 162),
 ('실력', 146),
 ('에', 144),
 ('의', 137),
 ('로', 96),
 ('게임', 76),
 ('가', 69),
 ('들', 67),
 ('에서', 66),
 ('으로', 63),
 ('은', 62),
 ('도', 61),
 ('하는', 61),
 ('한', 56),
 ('플레이', 49),
 ('는', 48),
 ('전', 39),
 ('과', 38),
 ('그', 37),
 ('정도', 37),
 ('있다', 37),
 ('적', 37),
 ('인', 37),
 ('것', 35),
 ('시즌', 33),
 ('때', 32),
 ('와', 32),
 ('할', 30),
 ('등', 29),
 ('이다', 29),
 ('년', 28),
 ('했다', 28),
 ('선수', 26),
 ('방송', 25),
 ('수', 25),
 ('한다', 25),
 ('월', 23),
 ('하고', 22),
 ('말', 22),
 ('까지', 21),
 ('못', 20),
 ('나', 20),
 ('최고', 20),
 ('일', 20),
 ('시청자', 20),
 ('실버', 20),
 ('잘', 19),
 ('다', 19),
 ('챔피언', 18),
 ('에게', 18),
 ('모습', 17),
 ('컬', 17),
 ('고', 17),
 ('위', 17),
 ('자신', 17),
 ('프로', 17),
 ('본인', 16),
 ('이나', 16),
 ('달성', 15),
 ('엄청난', 15),
 ('매우', 15),
 ('챌', 15),
 ('게', 15),
 ('저', 15),
 ('다른', 15),
 ('리그', 14),
 ('오브', 14),
 ('해', 14),
 ('하며', 14),
 ('천상계', 14),
 ('피지', 14),
 ('같은', 14),
 ('이후', 14),
 ('사람', 14),
 ('클리어', 14),
 ('때문', 14),
 ('여러', 13),

In [18]:
stop_words = ['을', '를','의','이','에','로','가','들','으로','은','도','에서','했다','것','인','는','와','있다','때','과','그','하고','한다','년','수','이다']

tokens_ko = [each_word for each_word in tokens_ko
                if each_word not in stop_words]

ko = nltk.Text(tokens_ko)
ko.vocab().most_common(100)

[('실력', 146),
 ('게임', 76),
 ('하는', 61),
 ('한', 56),
 ('플레이', 49),
 ('전', 39),
 ('정도', 37),
 ('적', 37),
 ('시즌', 33),
 ('할', 30),
 ('등', 29),
 ('선수', 26),
 ('방송', 25),
 ('월', 23),
 ('말', 22),
 ('까지', 21),
 ('못', 20),
 ('나', 20),
 ('최고', 20),
 ('일', 20),
 ('시청자', 20),
 ('실버', 20),
 ('잘', 19),
 ('다', 19),
 ('챔피언', 18),
 ('에게', 18),
 ('모습', 17),
 ('컬', 17),
 ('고', 17),
 ('위', 17),
 ('자신', 17),
 ('프로', 17),
 ('본인', 16),
 ('이나', 16),
 ('달성', 15),
 ('엄청난', 15),
 ('매우', 15),
 ('챌', 15),
 ('게', 15),
 ('저', 15),
 ('다른', 15),
 ('리그', 14),
 ('오브', 14),
 ('해', 14),
 ('하며', 14),
 ('천상계', 14),
 ('피지', 14),
 ('같은', 14),
 ('이후', 14),
 ('사람', 14),
 ('클리어', 14),
 ('때문', 14),
 ('여러', 13),
 ('하면', 13),
 ('자주', 13),
 ('롤', 13),
 ('능력', 13),
 ('\n', 13),
 ('티어', 13),
 ('라는', 13),
 ('서', 13),
 ('에는', 13),
 ('컨트롤', 13),
 ('라인', 12),
 ('점', 12),
 ('하지만', 12),
 ('하여', 12),
 ('상대', 12),
 ('맵', 12),
 ('시간', 12),
 ('중', 12),
 ('계속', 12),
 ('있는', 12),
 ('라운드', 11),
 ('시작', 11),
 ('뛰어난', 11),
 ('유저', 11),
 ('킬', 11),


In [19]:
i = 0
for sentence in sentence_df['sentence']:
    morph_sentence = t.morphs(sentence)
    clean_sentence = [each_word for each_word in morph_sentence
                if each_word not in stop_words]
    new_sentence = " ".join(clean_sentence)
    sentence_df['sentence'][i] = new_sentence
    i += 1

sentence_df.head(10)

,sentence,lable
0,카오스 아키에이지 리그 오브 레전드 오버 워치 배틀 라운드 등 해 가장 인기 끌었던...,1
1,어떤 게임 이든지 파고들기 시작 하면 최 상위 권 달성 하며 엄청난 실력 재능 충 ...,1
2,게임 센스 매우 뛰어난 편이 며 화려한 플레이 자주 나오는 편이 다,1
3,잿불 거인 나타나기 전 우디르 고인 챔이라는 인식 강해 천상계 아무 우디르 쓰지 않...,1
4,당연히 천상계 에선 원 탑 우디르 유저,1
5,오버 워치 비슷한 FPS 게임 재능 보이는 파카 답 게 솔로 스쿼드 가리지 않고 엄...,1
6,롤 PD 들이 파카 방송 모여 스쿼드 자주 하였는데 PD 한두 명 킬 킬 하 거나 ...,1
7,플레이 스타일 피지 컬 반응속도 찍어 누르는 재능 충의 표본,1
8,뛰어난 로지 컬 상황 판단력 각 재고 들어가는 타입 아니라 일단 몸 던지며 최대한 ...,1
9,상황 에서도 본인 피지 컬 신뢰 하며 절대 빼지 않고 몸 던진다,1


In [23]:
# 모델 저장

import pickle

with open('pipe_tfidf-2.dat', 'wb') as fp:
    pickle.dump(pipeline, fp)

In [105]:
# 새로운 문장 예측함수

def using_model(string):
    import numpy as np
    with open('pipe_tfidf.dat', 'rb') as fp:
        pipeline = pickle.load(fp)

    text = [string]
    

    # 예측 스코어
    r1 = np.max(pipeline.predict_proba(text) * 100)
    
    # 예측 결과
    r2 = pipeline.predict(text)[0]
    

    if r2 == 1 :
        print('실력 좋다')
    else :
        print('실력 나쁘다')

    print('스코어 : %.3f '% r1)


using_model('진짜 존나 못한다')

실력 나쁘다
스코어 : 56.659 
